In [1]:
import datetime as dt
from datetime import datetime
from secrets_p5 import secrets
import json
import pandas as pd
from sqlalchemy import create_engine, text
import requests

In [151]:
def execute_sql(**kwarg):
    pg_user = kwarg['pg_user']
    pg_password = kwarg['pg_password']
    pg_host = kwarg['pg_host']
    pg_port = kwarg['pg_port']
    pg_base_name = kwarg['pg_base_name']
    stage_schema_name = kwarg['stage_schema_name']
    dm_schema_name = kwarg['dm_schema_name']
    sql = kwarg['sql']
    sql = sql.format(stage_schema_name_f = stage_schema_name, dm_schema_name_f = dm_schema_name)
    engine = create_engine(
        f'postgresql://{pg_user}:{pg_password}@{pg_host}:{pg_port}/{pg_base_name}', 
        future=True)
    with engine.begin() as conn:
        result = conn.execute(text(sql))
        conn.commit()
    engine.dispose()

def get_data_from_api(**kwarg):
    api_key = kwarg['api_key']
    api_endpoint = kwarg['api_endpoint']
    nickname = kwarg['nickname']
    cohort = kwarg['cohort']
    api_method = kwarg['api_method']
    offset = 0
    headers = {'X-Nickname': nickname, 
               'X-Cohort': cohort, 
               'X-API-KEY': api_key
           }
    start_time = dt.datetime.now()
    while (dt.datetime.now() - start_time).total_seconds() < 30:
        res = requests.get('https://'+api_endpoint + api_method,
                           headers = headers, 
                           params = {'sort_field':'_id', 
                                     'sort_direction':'asc',
                                     'offset': offset,
                                     'limit': 30})
        res_json = json.loads(res.content)
        if len(res_json) == 0:
            break
        if offset == 0:
           df_all = pd.DataFrame(res_json)
        if offset > 0:
            df_new = pd.DataFrame(res_json)
            df_all = pd.concat([df_all, df_new], ignore_index=True)
        offset = offset + len(res_json)
    return df_all

In [152]:
op_kwargs = {
        'nickname': secrets['nickname'],
        'cohort': secrets['cohort'],
        'api_key': secrets['api_key'],
        'api_endpoint': secrets['api_endpoint'],
        'pg_host': secrets['pg_host'],
        'pg_port': secrets['pg_port'],
        'pg_base_name': secrets['pg_base_name'],
        'pg_user': secrets['pg_user'],
        'pg_password': secrets['pg_password'],
        'stage_schema_name': secrets['stage_schema_name'],
        'dm_schema_name': secrets['dm_schema_name'],
        'api_method': '/couriers'
        }

In [153]:
df_couriers = get_data_from_api(**op_kwargs)

[{'_id': '0d7u6nvgvx8ehsvyrelr6np', 'name': 'Алексей Иванов'}, {'_id': '0srjdu35in4sb1zycrfbax1', 'name': 'Елизавета Великая'}, {'_id': '0xwmzgvyi79og3mqxpg8xi5', 'name': 'Екатерина Алексеева'}, {'_id': '10xjhdn45az14zozlf1qcek', 'name': 'Оксана Волкова'}, {'_id': '1hrdkugw6oof2thvhomeovx', 'name': 'Екатерина Козлова'}, {'_id': '1j053zv7afiv1o6wky4j2o1', 'name': 'Алексей Михайлов'}, {'_id': '1mb5dv0328zbq5kz1g3zcjg', 'name': 'Алексей Васильев'}, {'_id': '1x1f1rc8yswpoxi2fx8cvjx', 'name': 'Александр Кузнецов'}, {'_id': '1z4wuqge11hm4792ojwjr6q', 'name': 'Анна Лебедева'}, {'_id': '21iyd66tad2x1as0j2myfxo', 'name': 'Иван Кузнецов'}, {'_id': '2il52qop5siu3kvtr0w8rjt', 'name': 'Иван Смирнов'}, {'_id': '2ohq565yafsb336ybnxbnwt', 'name': 'Петр Кузнецов'}, {'_id': '2qpfue45a1tr772c2yisper', 'name': 'Константин Попов'}, {'_id': '2rh4gruu0z4z3lv2j0h82dv', 'name': 'Петр Соколов'}, {'_id': '2y9b500v1nt7fxvkfwzx4jb', 'name': 'Светлана Великая'}, {'_id': '317ikmxh4oiy5pdu9dd86kr', 'name': 'Константи

In [156]:
df_couriers['_id'].duplicated().sum()

0

In [47]:
def get_data_from_api(**kwarg):
    api_key = kwarg['api_key']
    api_endpoint = kwarg['api_endpoint']
    nickname = kwarg['nickname']
    cohort = kwarg['cohort']
    api_method = kwarg['api_method']
    offset = 0
    headers = {'X-Nickname': nickname, 
               'X-Cohort': cohort, 
               'X-API-KEY': api_key
           }
    

    res = requests.get('https://'+api_endpoint + api_method,
                           headers = headers, 
                           params = {'sort_field':'id', 
                                     'sort_direction':'asc',
                                     'offset': offset})
    print(res)
    print(json.loads(res.content))